*Import modules*

In [200]:
import pandas as pd   #to create dataframe
import requests       #to send request to the URL
from bs4 import BeautifulSoup #to get the content in the form of HTML

In [201]:
url = 'https://www.imdb.com/search/title/?count=100&title_type=tv_series&genres=drama&ref_=adv_prv'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [202]:
rank = []
series_title = []
start_year = []
end_year = []
runtime = []
genre = []
rating = []
votes = []
description = []
actors = []

In [203]:
series_data = soup.findAll('div', attrs={'class' : 'lister-item mode-advanced'})

In [204]:
for data in series_data:
    rank_find = data.h3.find('span', class_ = 'lister-item-index unbold text-primary').text.replace('.','')
    rank.append(rank_find)
    
    title = data.h3.a.text
    series_title.append(title)

    year_released = data.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(','').replace(')','').strip()

    splitted = year_released.split('–')
    if len(splitted) > 1 :
        if splitted[1]!= '':
            start_year.append(splitted[0])
            end_year.append(splitted[1])
        else :
            start_year.append(splitted[0])
            end_year.append("Present")
    else :
        start_year.append(year_released)
        end_year.append(year_released)


    time_element = data.p.find('span', class_='runtime')
    if time_element is not None:
        time_find = time_element.text.replace(' min', '')
        runtime.append(time_find)
    else:
        runtime.append("None")

    genre_find = data.p.find('span', class_ = 'genre').text.replace('\n ','').strip().split(',')
    genre.append(genre_find)

    rating_find = data.find('div', class_ ='inline-block ratings-imdb-rating').text.replace('\n', '').strip()
    rating.append(rating_find)

    vote_find = data.find_all('span', attrs={'name':'nv'})
    votes.append(vote_find[0].text)

    desc_find = data.find_all('p', class_ = 'text-muted')
    description.append(desc_find[1].text.replace('\n', ''))

    actors_find = data.find('p', class_ = '').text.replace('\n', '').replace('Stars:','').strip().split(',')
    actors.append(actors_find)


In [205]:
data = {
    'Rank' : rank,
    'TV Series Title': series_title,
    'Year Started': start_year,
    'Year Ended': end_year,
    'Duration': runtime,
    'Genres': genre,
    'Rating': rating,
    'Votes': votes,
    'Description': description,
    'Actors': actors
}

newdata = pd.DataFrame(data)

In [206]:
newdata

,Rank,TV Series Title,Year Started,Year Ended,Duration,Genres,Rating,Votes,Description,Actors
0,1,The Witcher,2019,Present,60,"[Action, Adventure, Drama]",8.1,"532,263","Geralt of Rivia, a solitary monster hunter, st...","[Henry Cavill, Freya Allan, Anya Chalotra, ..."
1,2,Black Mirror,2011,Present,60,"[Drama, Mystery, Sci-Fi]",8.7,"598,168","An anthology series exploring a twisted, high-...","[Wunmi Mosaku, Monica Dolan, Daniel Lapaine,..."
2,3,The Bear,2022,Present,30,"[Comedy, Drama]",8.5,"112,276",A young chef from the fine dining world return...,"[Jeremy Allen White, Ebon Moss-Bachrach, Ayo..."
3,4,Silo,2023,Present,None,"[Drama, Mystery, Sci-Fi]",8.1,"56,619",Men and women live in a giant silo underground...,"[Rebecca Ferguson, Common, Tim Robbins, Har..."
4,5,Jack Ryan,2018,2023,60,"[Action, Drama, Thriller]",8.0,"155,570","Up-and-coming CIA analyst, Jack Ryan, is thrus...","[John Krasinski, Wendell Pierce, Michael Kel..."
...,...,...,...,...,...,...,...,...,...,...
95,96,The Vampire Diaries,2009,2017,43,"[Drama, Fantasy, Horror]",7.7,"333,727","The lives, loves, dangers and disasters in the...","[Nina Dobrev, Paul Wesley, Ian Somerhalder, ..."
96,97,The Last Kingdom,2015,2022,60,"[Action, Drama, History]",8.5,"155,838",As Alfred the Great defends his kingdom from N...,"[Alexander Dreymon, Eliza Butterworth, Arnas..."
97,98,Delete,2023,Present,45,"[Drama, Horror, Romance]",6.3,786,A complicated relationship with secrets to hid...,"[Natara Nopparatayapon, Nat Kitcharit, Sarik..."
98,99,Riverdale,2017,2023,45,"[Crime, Drama, Mystery]",6.5,"149,847",While navigating the troubled waters of romanc...,"[Lili Reinhart, Camila Mendes, Cole Sprouse,..."


In [207]:
newdata.isnull().sum()

Rank               0
TV Series Title    0
Year Started       0
Year Ended         0
Duration           0
Genres             0
Rating             0
Votes              0
Description        0
Actors             0
dtype: int64

In [208]:
newdata['Rank'].is_unique

True

In [209]:
newdata['TV Series Title'].is_unique

True

In [210]:
finaldata = newdata.to_json(orient='records', indent=2)

In [211]:
filepath = '/Users/rana/Downloads/finaldata.json'
with open(filepath, 'w') as json_file:
    json_file.write(finaldata)

In [212]:
path = '/Users/rana/Downloads/finaldata.csv'
newdata.to_csv(path, index=False)